# Imports

In [11]:
import urllib.request
import json
import csv
from google.colab import files

#Prompt User for Input and Print Results

In [12]:
#Prompt the user to input a magnitude paramter of type floating point
def getMagnitude():
  global userMagnitude
  magnitude = float(input("Please enter an earthquake magnitude (0.0 - 10.0): "))
  while True:
    if 0.0 <= magnitude <= 10.0:
      userMagnitude = magnitude
      return magnitude
    else:
      print("Invalid Input. Magnitude must be between 0.0 and 10.0")

#Provide a prompt to the user to end the program or input another magnitude number
def promptForEnd():
  while True:
    choice = input("Would you like to input another magnitude (Yes or No)? ").strip().capitalize()
    if choice == "Yes":
      return True
    elif choice == "No":
      return False
    else:
      print("Invalid Input. Please enter 'Yes' or 'No'.")

#Print results based on data and magnitude
def printResults(data, userMagnitude):
  jsonData = json.loads(data.decode("utf-8"))

  print(f"Metadata Title: {jsonData['metadata']['title']}")
  print(f"Number of Earthquake Events: {jsonData['metadata']['count']}")

  for feature in jsonData['features']:
    property = feature['properties']
    location = property['place']
    magnitude = property['mag']

    if magnitude > userMagnitude:
      print(f"Location: {location}, Magnitude: {magnitude}")

#Return the earthquake data in a list of dictionaries
def printResults2(data, userMagnitude):
  jsonData = json.loads(data.decode("utf-8"))
  earthquakeData = []

  for feature in jsonData['features']:
    properties = feature['properties']
    title = properties['title']
    place = properties['place']
    magnitude = properties['mag']
    time = properties['time']

    if magnitude > userMagnitude:
      earthquakeData.append({
          'Title': title,
          'Magnitude': magnitude,
          'Location': place,
          'Time': time
      })
  return earthquakeData

#Search based on a specified field
def printResults3(data, searchField, searchValue):
  jsonData = json.loads(data.decode("utf-8"))
  matchingResults = []

  for feature in jsonData['features']:
    properties = feature['properties']
    if searchField in properties and properties[searchField] == searchValue:
      title = properties['title']
      magnitude = properties['mag']
      place = properties['place']
      time = properties['time']
      matchingResults.append({
          "Title": title,
          "Magnitude": magnitude,
          "Location": place,
          "Time": time
      })

  for result in matchingResults:
    print(f"Title: {result['Title']} | ", end = "")
    print(f"Magnitude: {result['Magnitude']} | ", end = "")
    print(f"Location: {result['Location']} | ", end = "")
    print(f"Time: {result['Time']}")

# Get Earthquake Data

In [13]:
#Get earthquake data from the US Geological Services API
def getData():
  api_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"

  respone = urllib.request.urlopen(api_url)

  statusCode = response.getcode()
  print(f"HTTPS Status Code: {statusCode}")

  if statusCode == 200:
    data = response.read()
    userMagnitude = getMagnitude()
    printResults(data, userMagnitude)
  else:
    print(f"Error Non-Valid Status Code: {statusCode}")

# Output Data to Spreadsheet

In [14]:
#Save earthquake data to a CSV file
def saveCSV(earthquakeData, filename = "earthquakeData.csv"):
  fieldnames = ['Title', 'Magnitude', 'Location', 'Time']
  with open(filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in earthquakeData:
      writer.writerow(row)

  print(f"Data successfully saved to filename: {filename}")
  files.download(filename)

# Main Function

In [15]:
def main():
  while True:
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"
    response = urllib.request.urlopen(url)
    data = response.read()

    userMagnitude = getMagnitude()
    earthquakeData = printResults2(data, userMagnitude)
    saveCSV(earthquakeData)
    while True:
      search = input("Would you like to search for a specific field (Yes or No)? ").capitalize()
      if search == "Yes":
        searchField = input("Enter the field you want to seach (ex. 'place' or 'mag'): ")
        searchValue = input(f"Enter the value you want to search for in {searchField}: ")
        printResults3(data, searchField, searchValue)
      elif search == "No":
        break
      else:
        print("Invalid input. Please enter either 'Yes' or 'No'.")
    if not promptForEnd():
      break

main()

Please enter an earthquake magnitude (0.0 - 10.0): 7.2
Data successfully saved to filename: earthquakeData.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Would you like to search for a specific field (Yes or No)? Yes
Enter the field you want to seach (ex. 'place' or 'mag'): mag
Enter the value you want to search for in mag: 6.4
Would you like to search for a specific field (Yes or No)? No
Would you like to input another magnitude (Yes or No)? No
